# Creating a Face Swap App

In this project, we will utilize OpenCV and Dlib to detect and extract human faces from a given image. A pre-trained model will be employed to identify facial landmarks for precise face detection and processing.

In [2]:
# Importing necessary libraries
import cv2  # For image processing and computer vision tasks
import numpy as np  # For numerical operations and matrix manipulations
import dlib  # For face detection and landmark extraction
import requests  # For handling HTTP requests (e.g., downloading images)
from PIL import Image  # For advanced image processing and handling


In [1]:
import os
import requests

def download_shape_predictor(destination="shape_predictor_68_face_landmarks.dat"):
    """Downloads the Dlib shape predictor model if not already present."""
    url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
    compressed_file = destination + ".bz2"

    # Check if the file already exists
    if os.path.exists(destination):
        print(f"{destination} already exists. Skipping download.")
        return

    print(f"Downloading {destination}...")
    try:
        # Download the compressed file
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(compressed_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print("Download complete. Extracting file...")

        # Extract the .bz2 file
        import bz2
        with bz2.BZ2File(compressed_file, "rb") as bz_file:
            with open(destination, "wb") as out_file:
                out_file.write(bz_file.read())

        print(f"{destination} successfully extracted.")
        # Optionally delete the compressed file
        os.remove(compressed_file)
    except Exception as e:
        print(f"An error occurred: {e}")

# Download the shape predictor
download_shape_predictor()


Download complete. Extracting file...
shape_predictor_68_face_landmarks.dat successfully extracted.


In [4]:
def extract_index_nparray(nparray):
    """
    Extracts the first element from the first row of a NumPy array.

    Parameters:
        nparray (np.ndarray): A NumPy array with at least one row and one element.

    Returns:
        int or None: The first element of the array, or None if the array is empty.
    """
    if nparray.size > 0:
        return nparray[0, 0]
    return None


Next, we will define a function to retrieve the index from a NumPy array.

In [6]:
from PIL import Image
import requests

def fetch_and_resize_image(url, size=(300, 300)):
    """
    Fetches an image from the given URL, resizes it, and returns the processed image.

    Parameters:
        url (str): The URL of the image to fetch.
        size (tuple): The target size for resizing the image (width, height).

    Returns:
        PIL.Image.Image: The resized image.
    """
    try:
        # Fetch the image from the URL
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Open and resize the image
        image = Image.open(response.raw)
        image = image.resize(size)
        return image
    except requests.RequestException as e:
        print(f"Error fetching the image: {e}")
    except Exception as e:
        print(f"Error processing the image: {e}")
    return None

# Example usage
image_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSx8Pu1tW1uCiZPfj9K1EL6uHxbg3bOKO9XkA&usqp=CAU'
image1 = fetch_and_resize_image(image_url)
if image1:
    image1.show()  # Display the image (optional)


Next, we will fetch our source image from the internet using its URL and resize it to the desired dimensions.

In [7]:
from PIL import Image
import requests

def fetch_and_resize_image(url, size=(300, 300)):
    """
    Fetches an image from the given URL, resizes it to the specified size, and returns the image.

    Parameters:
        url (str): The URL of the image to fetch.
        size (tuple): The target size for resizing the image (width, height).

    Returns:
        PIL.Image.Image: The resized image or None if there was an error.
    """
    try:
        # Send a request to get the image from the URL
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Ensure we received a valid response

        # Open and resize the image
        image = Image.open(response.raw)
        image = image.resize(size)
        return image
    except requests.RequestException as e:
        print(f"Error fetching the image: {e}")
    except Exception as e:
        print(f"Error processing the image: {e}")
    return None

# Example usage
image_url2 = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYX1dyl9INRo5cbvDeTILRcZVzfcMsCsE0kg&usqp=CAU'
image2 = fetch_and_resize_image(image_url2)
if image2:
    image2.show()  # Optionally display the image


Here, we will load the destination image from the internet using its URL and resize it to the desired dimensions.

In [10]:
import numpy as np
import cv2
from PIL import Image

def convert_to_gray(image):
    """
    Converts a PIL Image to a NumPy array and then to grayscale.

    Parameters:
        image (PIL.Image.Image): The input image to be converted.

    Returns:
        np.ndarray: The grayscale image as a NumPy array.
    """
    # Convert PIL Image to NumPy array
    img_array = np.array(image)

    # Convert the image to grayscale
    img_gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)

    return img_array, img_gray

# Convert both images (image1 and image2) to arrays and grayscale
img1_array, img1_gray = convert_to_gray(image1)
img2_array, img2_gray = convert_to_gray(image2)

# Create an empty mask for the first image
mask = np.zeros_like(img1_gray)


Now, we will convert our images into NumPy arrays and apply OpenCV to convert them into grayscale. Additionally, we will create an empty mask with the same shape as our source image, initialized with zeros.

Here we will first load Face detector and Face landmarks predictor using dlib and then we will find the height, width, channels which are required for creating empty image with zeros.

In [ ]:
import cv2
import numpy as np

def extract_index_from_points(points, pt):
    """
    Extracts the index of a point from the given array of points.

    Parameters:
        points (np.ndarray): The array of points.
        pt (tuple): The point to find in the array.

    Returns:
        int or None: The index of the point, or None if the point is not found.
    """
    # Use np.where to find the index of the point in the array
    index = np.where((points == pt).all(axis=1))
    return extract_index_nparray(index) if index[0].size > 0 else None

# Face 1
faces = detector(img_gray)
for face in faces:
    landmarks = predictor(img_gray, face)

    # Extract landmark points
    landmarks_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(0, 68)]

    points = np.array(landmarks_points, np.int32)
    convexhull = cv2.convexHull(points)
    cv2.fillConvexPoly(mask, convexhull, 255)

    # Masking the face region
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)

    # Delaunay triangulation
    rect = cv2.boundingRect(convexhull)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks_points)
    triangles = np.array(subdiv.getTriangleList(), dtype=np.int32)

    indexes_triangles = []
    for t in triangles:
        pt1, pt2, pt3 = (tuple(t[i:i+2]) for i in range(0, len(t), 2))

        # Extract indices of triangle points
        index_pt1 = extract_index_from_points(points, pt1)
        index_pt2 = extract_index_from_points(points, pt2)
        index_pt3 = extract_index_from_points(points, pt3)

        # If all triangle points are found, add the triangle to the list
        if None not in [index_pt1, index_pt2, index_pt3]:
            indexes_triangles.append([index_pt1, index_pt2, index_pt3])


First, we pass the image to the face detector, which is then used to extract landmarks using the shape predictor. The extracted landmark points (x and y coordinates) are stored in a list. Next, we segment the face into triangles. This step is essential for the face-swapping process, as each triangle will later be swapped with the corresponding triangle from the destination image. The triangulation of the destination image must match the pattern of the source image, meaning the connections between points must be identical. Therefore, after triangulating the source image, we extract the indices of the landmark points, which we use to replicate the same triangulation on the destination image. Once we have the triangle indices, we loop through them to triangulate the destination face.

In [ ]:
# Face 2
faces2 = detector(img2_gray)
for face in faces2:
    # Extract landmarks for the second face
    landmarks = predictor(img2_gray, face)
    landmarks_points2 = []

    # Loop through all 68 landmarks
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points2.append((x, y))

    # Convert landmarks to a NumPy array
    points2 = np.array(landmarks_points2, dtype=np.int32)

    # Create a convex hull for the second set of landmarks
    convexhull2 = cv2.convexHull(points2)



Next, we will apply a similar process as we did for the source image to extract landmarks from the destination image.

In [ ]:
# Creating empty masks
source_face_mask = np.zeros_like(img_gray)  # Mask for the source image
destination_face_mask = np.zeros_like(img2)  # Mask for the destination image


We will create empty images filled with zeros, which will be used for further processing.

In [ ]:
# Triangulation of both faces
for triangle_index in indexes_triangles:
    # Extracting points for the first face
    pt1_1 = landmarks_points[triangle_index[0]]
    pt2_1 = landmarks_points[triangle_index[1]]
    pt3_1 = landmarks_points[triangle_index[2]]
    triangle1 = np.array([pt1_1, pt2_1, pt3_1], np.int32)

    # Bounding rectangle and mask for the first face triangle
    rect1 = cv2.boundingRect(triangle1)
    (x1, y1, w1, h1) = rect1
    cropped_triangle1 = img[y1: y1 + h1, x1: x1 + w1]
    mask_triangle1 = np.zeros((h1, w1), np.uint8)

    points1 = np.array([
        [pt1_1[0] - x1, pt1_1[1] - y1],
        [pt2_1[0] - x1, pt2_1[1] - y1],
        [pt3_1[0] - x1, pt3_1[1] - y1]
    ], np.int32)
    cv2.fillConvexPoly(mask_triangle1, points1, 255)

    # Visualizing triangles on the source image
    cv2.line(source_face_mask, pt1_1, pt2_1, 255)
    cv2.line(source_face_mask, pt2_1, pt3_1, 255)
    cv2.line(source_face_mask, pt1_1, pt3_1, 255)

    # Extracting points for the second face
    pt1_2 = landmarks_points2[triangle_index[0]]
    pt2_2 = landmarks_points2[triangle_index[1]]
    pt3_2 = landmarks_points2[triangle_index[2]]
    triangle2 = np.array([pt1_2, pt2_2, pt3_2], np.int32)

    # Bounding rectangle and mask for the second face triangle
    rect2 = cv2.boundingRect(triangle2)
    (x2, y2, w2, h2) = rect2
    mask_triangle2 = np.zeros((h2, w2), np.uint8)

    points2 = np.array([
        [pt1_2[0] - x2, pt1_2[1] - y2],
        [pt2_2[0] - x2, pt2_2[1] - y2],
        [pt3_2[0] - x2, pt3_2[1] - y2]
    ], np.int32)
    cv2.fillConvexPoly(mask_triangle2, points2, 255)

    # Affine transformation from source triangle to destination triangle
    points1 = np.float32(points1)
    points2 = np.float32(points2)
    transform_matrix = cv2.getAffineTransform(points1, points2)
    warped_triangle = cv2.warpAffine(cropped_triangle1, transform_matrix, (w2, h2))
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangle2)

    # Overlaying the warped triangle on the destination face
    destination_face_area = img2_new_face[y2: y2 + h2, x2: x2 + w2]
    destination_face_area_gray = cv2.cvtColor(destination_face_area, cv2.COLOR_BGR2GRAY)
    _, mask_inverse = cv2.threshold(destination_face_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_inverse)

    destination_face_area = cv2.add(destination_face_area, warped_triangle)
    img2_new_face[y2: y2 + h2, x2: x2 + w2] = destination_face_area



Here, we perform similar operations for the destination image as we did for the source image. Once we have the triangulation for both faces, we extract the triangles from the source face. We also determine the coordinates of the corresponding triangles in the destination face. This allows us to warp the source face triangles to match the size and perspective of the corresponding triangles on the destination face.

In [ ]:
# Face swapping: Embedding the first face onto the second face
face_mask = np.zeros_like(img2_gray)
head_mask = cv2.fillConvexPoly(face_mask, convexhull2, 255)
face_mask_inverse = cv2.bitwise_not(head_mask)



Once all the triangles have been cut and warped, the next step is to assemble them. We reconstruct the face using the same triangulation pattern, but this time, we replace each triangle with its warped counterpart.

In [ ]:
# Removing the original face from the second image and replacing it with the swapped face
head_without_face = cv2.bitwise_and(img2, img2, mask=face_mask_inverse)
final_result = cv2.add(head_without_face, img2_new_face)


The face is now prepared for replacement. We remove the face from the destination image to create space for the new face. Then, we combine the new face with the destination image without its original face.

In [ ]:
# Creating a seamless clone of the two faces
(x, y, w, h) = cv2.boundingRect(convexhull2)
center_of_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))
seamless_face_swap = cv2.seamlessClone(result, img2, face_mask, center_of_face2, cv2.NORMAL_CLONE)


Finally, the faces are successfully swapped, and it's time to adjust the colors to ensure the source face blends seamlessly with the destination image. OpenCV provides a built-in function called seamlessClone that automatically handles this operation. To apply it, we take the new face (created in the 6th step), use the original destination image and its mask to cut out the face, and determine the center of the face for proper alignment.

In [ ]:
# Converting the array back to an image
Image.fromarray(seamless_face_swap)


Finally, we will visualize the output by converting the numpy array into a Pillow Image object.

# Conclusion

We began by downloading a pretrained model for face landmarks and images from the internet to work on. Then, we utilized OpenCV and Dlib for image preprocessing, applying various functions to ultimately achieve the goal of swapping the face from the source image onto the destination image.

# Scope

This project serves as an excellent resource for learning and understanding key concepts in computer vision. It can also be applied to build augmented reality applications, such as Snapchat, where face swapping and similar effects are commonly used.